In [1]:
import pandas as pd

# File paths
sales_file_path = "/Users/tiyang/Downloads/MGTA452_Midterm2023/sales_yogurt.csv"
stores_file_path = "/Users/tiyang/Downloads/MGTA452_Midterm2023/stores_yogurt.csv"
products_file_path = "/Users/tiyang/Downloads/MGTA452_Midterm2023/products_yogurt.csv"
extras_file_path = "/Users/tiyang/Downloads/MGTA452_Midterm2023/extras.csv"

# Load datasets
sales_df = pd.read_csv(sales_file_path)
stores_df = pd.read_csv(stores_file_path)
products_df = pd.read_csv(products_file_path)
extras_df = pd.read_csv(extras_file_path)

# Show the first few rows of each dataset
sales_head = sales_df.head()
stores_head = stores_df.head()
products_head = products_df.head()
extras_head = extras_df.head()

sales_head, stores_head, products_head, extras_head


(   storeID  WEEK    product  s1     s2
 0   234212  1583   24667510  30  20.10
 1   234212  1583   24667580  27  18.09
 2   234212  1583   24667579  36  24.12
 3   234212  1583  181146310   2   7.18
 4   234212  1583  181146300   7  26.13,
    storeID        Market_Name
 0   200059       PHILADELPHIA
 1   200197  PEORIA/SPRINGFLD.
 2   200272        LOS ANGELES
 3   200297        PORTLAND,OR
 4   200341          SAN DIEGO,
         P0           P1     P2                    P3       P4
 0  CHOBANI  81947001005  0.375             BLUEBERRY   NONFAT
 1  CHOBANI  81947001051  1.000  CHOCOLATE & HONEYNAN  LOW FAT
 2  CHOBANI  81947001003  0.375                 HONEY   NONFAT
 3  CHOBANI  81947001010  0.375             PINEAPPLE  LOW FAT
 4  CHOBANI  81947001006  0.375                 PEACH   NONFAT,
      product    X1
 0  170470300  0.47
 1  236632260  2.62
 2  170470650  0.48
 3  170470307  0.54
 4  170470313  0.57)

In [2]:
# Step 1: Aggregate Dollar Sales
total_sales_per_store = sales_df.groupby('storeID')['s2'].sum()

# Step 2: Identify the Top Store
top_store_sales = total_sales_per_store.idxmax(), total_sales_per_store.max()

# Step 3: Report the Answer (rounded to nearest integer)
top_store_sales_rounded = (top_store_sales[0], round(top_store_sales[1]))
top_store_sales_rounded


(268492, 445003)

In [3]:
# Step 1: Merge Datasets
sales_with_brands = sales_df.merge(products_df[['P1', 'P0']], left_on='product', right_on='P1', how='left')

# Step 2: Aggregate Dollar Sales by Brand
total_sales_per_brand = sales_with_brands.groupby('P0')['s2'].sum()

# Step 3: Identify the Top Brand
top_brand_sales = total_sales_per_brand.idxmax(), total_sales_per_brand.max()

# Step 4: Report the Answer (rounded to nearest integer)
top_brand_sales_rounded = (top_brand_sales[0], round(top_brand_sales[1]))
top_brand_sales_rounded


('PRIVATE LABEL', 8568686)

In [4]:
# Step 1: Merge Datasets
sales_with_brands_and_volume = sales_df.merge(products_df[['P1', 'P0', 'P2']], left_on='product', right_on='P1', how='left')

# Step 2: Calculate Sales Volume in 16 oz Units
sales_with_brands_and_volume['sales_volume_16oz'] = sales_with_brands_and_volume['s1'] * sales_with_brands_and_volume['P2']

# Step 3: Aggregate Sales Volume by Brand
total_volume_per_brand = sales_with_brands_and_volume.groupby('P0')['sales_volume_16oz'].sum()

# Step 4: Identify the Top Brand
top_brand_volume = total_volume_per_brand.idxmax(), total_volume_per_brand.max()

# Step 5: Report the Answer
top_brand_volume


('PRIVATE LABEL', 7166144.3125)

In [5]:
# Step 1: Merge Datasets
sales_with_flavors = sales_df.merge(products_df[['P1', 'P3']], left_on='product', right_on='P1', how='left')

# Step 2: Aggregate Dollar Sales by Product
total_sales_per_product = sales_with_flavors.groupby(['product', 'P3'])['s2'].sum()

# Step 3: Identify the Top 10 Products
top_10_products = total_sales_per_product.sort_values(ascending=False).head(10)

# Step 4: Count Strawberry Flavored Products
strawberry_flavored_products_count = top_10_products[top_10_products.index.get_level_values('P3').str.contains('STRAWBERRY')].count()
strawberry_flavored_products_count


4

In [6]:
# Step 1: Filter for YOPLAIT ORIGINAL
yoplait_original_products = products_df[products_df['P0'].str.contains('YOPLAIT ORIGINAL', case=False, na=False)]

# Step 2: Merge Sales and Product Data
sales_yoplait_original = sales_df[sales_df['product'].isin(yoplait_original_products['P1'])]
sales_yoplait_original = sales_yoplait_original.merge(yoplait_original_products[['P1', 'P2']], left_on='product', right_on='P1', how='left')

# Step 3: Merge with Store Information
sales_yoplait_original = sales_yoplait_original.merge(stores_df, on='storeID', how='left')

# Step 4: Calculate Price per 16 oz
sales_yoplait_original['price_per_16oz'] = sales_yoplait_original['s2'] / (sales_yoplait_original['s1'] * sales_yoplait_original['P2'])

# Step 5: Calculate Average Price per 16 oz by Market
average_price_per_16oz_by_market = sales_yoplait_original.groupby('Market_Name')['price_per_16oz'].mean()

# Step 6: Identify Markets with Highest and Lowest Average Price
market_highest_average_price = average_price_per_16oz_by_market.idxmax()


market_highest_average_price


'NEW YORK'

In [7]:
# Provided options for the market with the lowest average price per 16oz
options_lowest_price_market = [
    "ATLANTA", "NEW ORLEANS, LA", "NEW YORK", "SACRAMENTO",
    "DETROIT", "CHICAGO", "TOLEDO", "TULSA,OK"
]

# Filter the average price per 16oz data to the provided options
average_price_filtered_options = average_price_per_16oz_by_market[average_price_per_16oz_by_market.index.isin(options_lowest_price_market)]

# Identify the market with the lowest average price per 16oz
market_lowest_average_price_filtered = average_price_filtered_options.idxmin()
market_lowest_average_price_filtered


'TOLEDO'

In [8]:
# Step 1: Filter for YOPLAIT ORIGINAL
yoplait_original_products = products_df[products_df['P0'].str.contains('YOPLAIT ORIGINAL', case=False, na=False)]

# Step 2: Merge Sales and Product Data
sales_yoplait_original = sales_df[sales_df['product'].isin(yoplait_original_products['P1'])]
sales_yoplait_original = sales_yoplait_original.merge(yoplait_original_products[['P1', 'P2']], left_on='product', right_on='P1', how='left')

# Step 3: Merge with Store Information
sales_yoplait_original = sales_yoplait_original.merge(stores_df, on='storeID', how='left')

# Step 4: Aggregate Dollar Sales by Market
total_sales_per_market = sales_yoplait_original.groupby('Market_Name')['s2'].sum()

# Step 5: Calculate Total Dollar Sales for YOPLAIT ORIGINAL
total_dollar_sales_yoplait_original = total_sales_per_market.sum()

# Step 6: Calculate Market Share
market_share = total_sales_per_market / total_dollar_sales_yoplait_original

# Step 7: Identify Market with Highest Market Share
market_highest_share = market_share.idxmax(), market_share.max()

# Format the result to two digits
market_highest_share_formatted = (market_highest_share[0], round(market_highest_share[1], 2))
market_highest_share_formatted


('LOS ANGELES', 0.12)

In [9]:
# Step 1: Merge Sales Data and Wholesale Prices
sales_with_cost = sales_df.merge(extras_df, left_on='product', right_on='product', how='left')

# Step 2: Calculate Total Cost and Profit
sales_with_cost['total_cost'] = sales_with_cost['X1'] * sales_with_cost['s1']
sales_with_cost['profit'] = sales_with_cost['s2'] - sales_with_cost['total_cost']

# Step 3: Aggregate Profit by Product
total_profit_per_product = sales_with_cost.groupby('product')['profit'].sum()

# Step 4: Identify the Product with the Highest Total Profit
highest_profit_product = total_profit_per_product.idxmax(), total_profit_per_product.max()

# Step 5: Report the Total Profit (rounded to nearest integer)
highest_profit_product_rounded = (highest_profit_product[0], round(highest_profit_product[1]))
highest_profit_product_rounded


(170470300, 154135)

In [10]:
# Step 1: Merge Sales and Product Data
sales_with_fat_content = sales_df.merge(products_df[['P1', 'P4', 'P2']], left_on='product', right_on='P1', how='left')

# Step 2: Calculate Price per 16 oz
sales_with_fat_content['price_per_16oz'] = sales_with_fat_content['s2'] / (sales_with_fat_content['s1'] * sales_with_fat_content['P2'])

# Step 3: Filter for Selected Fat Contents
selected_fat_contents = ["LOW FAT", "NONFAT", "1% LOWFAT"]
sales_selected_fat_contents = sales_with_fat_content[sales_with_fat_content['P4'].isin(selected_fat_contents)]

# Step 4: Calculate Average Price per 16 oz by Fat Content
average_price_per_16oz_by_fat_content = sales_selected_fat_contents.groupby('P4')['price_per_16oz'].mean()

# Step 5: Rank the Prices
price_rank = average_price_per_16oz_by_fat_content.sort_values(ascending=False)
price_rank


P4
LOW FAT           inf
NONFAT       2.057267
1% LOWFAT    1.284094
Name: price_per_16oz, dtype: float64